# Gesture Recognition
 build a 3D Conv model that will be able to predict the 5 gestures correctly. 
import required libraries

In [23]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [24]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [25]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 30 #experiment with the batch size

## Generator
This is one of the most important part of the code.In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [27]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = np.round(np.linspace(0,total_frames-1,frames_to_sample)).astype(int) #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image_resized=imresize(image,(image_height,image_width,3))
                    
                    batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        import cv2
        seq_len=len(t)%batch_size
        if (seq_len!=0):
            batch_data = np.zeros((seq_len,len(img_idx),image_height,image_width,channels)) 
            batch_labels = np.zeros((seq_len,num_classes)) 

            if (augment): batch_data_aug = np.zeros((seq_len,len(img_idx),image_height,image_width,channels))


            for folder in range(seq_len): 
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) 
                for idx,item in enumerate(img_idx): 
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image_resized=imresize(image,(image_height,image_width,3))


                    batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255
                    batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])/255
                    batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])/255

                    if (augment):
                        shifted = cv2.warpAffine(image, 
                                                 np.float32([[1, 0, np.random.randint(-30,30)],[0, 1, np.random.randint(-30,30)]]), 
                                                (image.shape[1], image.shape[0]))

                        gray = cv2.cvtColor(shifted,cv2.COLOR_BGR2GRAY)

                        x0, y0 = np.argwhere(gray > 0).min(axis=0)
                        x1, y1 = np.argwhere(gray > 0).max(axis=0) 

                        cropped=shifted[x0:x1,y0:y1,:]

                        image_resized=imresize(cropped,(image_height,image_width,3))

                        batch_data_aug[folder,idx,:,:,0] = (image_resized[:,:,0])/255
                        batch_data_aug[folder,idx,:,:,1] = (image_resized[:,:,1])/255
                        batch_data_aug[folder,idx,:,:,2] = (image_resized[:,:,2])/255


                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1


            if (augment):
                batch_data=np.concatenate([batch_data,batch_data_aug])
                batch_labels=np.concatenate([batch_labels,batch_labels])
            yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [28]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 6 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 6


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [29]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D,MaxPooling2D,Conv2D
from keras.layers.recurrent import LSTM
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

from keras.layers import Dropout
#write your model here

In [30]:
# Method to initialize batch size,epoch,image resolution,frame to sample
total_frames=30
frames_to_sample =20
image_height=100
image_width=100
channels=3
num_classes=5
x=frames_to_sample
y=image_height
z=image_width
augment=False
def initializeParemeters(frames,img_height,img_width,batchSize,epoch,aug=True):
    global frames_to_sample
    global image_height
    global image_width
    global num_epochs
    global batch_size
    global x
    global y
    global z
    global augment
    frames_to_sample=frames
    image_height=img_height
    image_width=img_width
    num_epochs=epoch
    batch_size=batchSize
    x=frames_to_sample
    y=image_height
    z=image_width
    augment=aug

# Conv3D Model 

In [31]:
def Conv3DModel():
    model = Sequential()
    model.add(Conv3D(16, (3, 3, 3), padding='same',input_shape=(frames_to_sample,image_height,image_width,channels)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))

    model.add(Conv3D(32, (2, 2, 2), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))

    model.add(Conv3D(64, (2, 2, 2), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))

    model.add(Conv3D(128, (2, 2, 2), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))

    model.add(Flatten())
    model.add(Dense(64,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(64,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))


    model.add(Dense(num_classes,activation='softmax'))
    return model


### 1 : Conv3D Model with 30 fames,100*100 image resolution,batchsize=40,epoch=15,augment = False
categorical_accuracy: 0.9544 - val_loss: 0.5788 - val_categorical_accuracy: 0.8083 ,Trainable params: 387,749

In [32]:
# initializeParemeters(30,100,100,40,15,False)
# model=Conv3DModel()

### 2 : Conv3D Model with 20 fames,100*100 image resolution,batchsize=30,epoch=15,augment = True
categorical_accuracy: 0.8849 - val_loss: 0.6062 - val_categorical_accuracy: 0.7636 , Trainable params: 387,749

In [33]:
# initializeParemeters(20,100,100,30,15,True)
# model=Conv3DModel()

# CNN-LSTM Model

In [34]:
def CNNLstm_model(lstm_cells=64,dense_neurons=64,dropout=0.25):

        model = Sequential()

        model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                                  input_shape=(frames_to_sample,image_height,image_width,channels)))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        model.add(TimeDistributed(Flatten()))

        model.add(LSTM(lstm_cells))
        model.add(Dropout(dropout))
        
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(Dropout(dropout))
        
        model.add(Dense(num_classes, activation='softmax'))
        
        return model

### 3:CNN - LSTM Model with 30 fames,124*124 image resolution,batchsize=30,epoch=10,augment=True
#### gru_cells=64,dense_neurons=64,dropout=0.25
categorical_accuracy: 0.7404 - val_loss: 1.1308 - val_categorical_accuracy: 0.5091

In [35]:
# initializeParemeters(30,124,124,30,10)
# model= CNNLstm_model()

### 4:CNN - LSTM Model with 20 fames,100*100 image resolution,batchsize=30,epoch=4,augment=True
#### gru_cells=64,dense_neurons=64,dropout=0.25
categorical_accuracy: 0.5305 - val_loss: 1.0920 - val_categorical_accuracy: 0.5727

In [36]:
# initializeParemeters(20,100,100,30,4)
# model= CNNLstm_model()

# Mobilenet +Gru :- Transfer Learning with GRU and training all weights

In [37]:
from keras.applications import mobilenet
mobilenet_transfer = mobilenet.MobileNet(input_shape=None,weights='imagenet', include_top=False)

def GRU_model(gru_cells=64,dense_neurons=64,dropout=0.25):
    
    model = Sequential()
    model.add(TimeDistributed(mobilenet_transfer,input_shape=(frames_to_sample,image_height,image_width,channels)))


    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Flatten()))

    model.add(GRU(gru_cells))
    model.add(Dropout(dropout))

    model.add(Dense(dense_neurons,activation='relu'))
    model.add(Dropout(dropout))

    model.add(Dense(num_classes, activation='softmax'))
    return model


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras_applications/mobilenet.py:206: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


### 5 : GRU Model with 25 fames,100*100 image resolution,batchsize=30,epoch=4,augment=True

#### gru_cells=64,dense_neurons=64,dropout=0.25
ResourceExhaustedError

### 6 : GRU Model with 20 fames,100*100 image resolution,batchsize=30,epoch=5,augment=True

#### gru_cells=64,dense_neurons=64,dropout=0.25
categorical_accuracy: 0.9448 - val_loss: 0.4075 - val_categorical_accuracy: 0.8364 ,Trainable params: 3,422,597


In [38]:
# initializeParemeters(20,100,100,30,5)
# model=GRU_model()

### 7 : GRU Model with 20 fames,100*100 image resolution,batchsize=30,epoch=10,augment=True
#### gru_cells=64,dense_neurons=64,dropout=0.25
categorical_accuracy: 0.9599 - val_loss: 0.2934 - val_categorical_accuracy: 0.9091,Trainable params: 3,422,597

In [39]:
initializeParemeters(20,100,100,30,10)
model=GRU_model()

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [40]:
optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_5 (TimeDist (None, 20, 3, 3, 1024)    3228864   
_________________________________________________________________
time_distributed_6 (TimeDist (None, 20, 3, 3, 1024)    4096      
_________________________________________________________________
time_distributed_7 (TimeDist (None, 20, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, 20, 1024)          0         
_________________________________________________________________
gru_2 (GRU)                  (None, 64)                209088    
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [41]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [42]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [43]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [44]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 30
Source path =  ./Project_data/train ; batch size = Epoch 1/10
30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/

23/23 [==============================] - 63s 3s/step - loss: 1.3642 - categorical_accuracy: 0.4376 - val_loss: 0.9049 - val_categorical_accuracy: 0.6727

Epoch 00001: saving model to model_init_2021-02-0813_35_36.208507/model-00001-1.36072-0.43544-0.90493-0.67273.h5
Epoch 2/10
23/23 [==============================] - 47s 2s/step - loss: 0.7074 - categorical_accuracy: 0.7467 - val_loss: 0.6974 - val_categorical_accuracy: 0.7182

Epoch 00002: saving model to model_init_2021-02-0813_35_36.208507/model-00002-0.70338-0.75526-0.69740-0.71818.h5
Epoch 3/10
23/23 [==============================] - 50s 2s/step - loss: 0.3841 - categorical_accuracy: 0.8824 - val_loss: 0.3389 - val_categorical_accuracy: 0.8455

Epoch 00003: saving model to model_init_2021-02-0813_35_36.208507/model-00003-0.39346-0.87838-0.33886-0.84545.h5
Epoch 4/10
23/23 [==============================] - 51s 2s/step - loss: 0.2238 - categorical_accuracy: 0.9376 - val_loss: 0.3859 - val_categorical_accuracy: 0.8000

Epoch 00004: